Computational Theory Problems

In [1]:
import numpy as np

Problem 1: Binary Words and Operations

In [ ]:
def Parity(x, y, z):
    """
    Return Parity function of three 32-bit words

    Parity(x, y, z) = x XOR y XOR z
    """
    return np.uint32(x ^ y ^ z)

In [ ]:
def Ch(x, y, z):
    """
    Apply Choose (CH) function on three 32-bit words

    Combines inputs using bitwise AND and XOR operations and returns a 32-bit result.
    For each bit position, result is selected from y, if corresponding bit of x is 1, otherwise z is selected
    """
    return np.uint32((x & y) ^ (~x & z))

In [ ]:
def Maj(x, y, z):
    """
    Apply Majority (MJ) function to three 32-bit words

    For each bit position, if output bit is 1, two or more corresponding bits of x, y and z are 1
    """
    return np.uint32((x & y) ^ (x & z) ^ (y & z))

In [ ]:
def ROTR(x, n, w):
    """
    Circular right rotation

    x is rotated right by n bits within the width of w
    """
    return np.uint32((x >> n) | (x << (w - n )))

In [ ]:
def Sigma0(x):
    """
    Sigma0 transformation to 32-bit value

    Rotates input to the right by a fixed number and returns the combined result using bitwise XOR
    """
    x = np.uint32(x)

    return np.uint32(ROTR(x, 2) ^ ROTR(x, 13) ^ ROTR(x, 22))

In [ ]:
def Sigma1(x):
    """
    Sigma1 transformation to a 32-bit value

    Rotates input to the right by a fixed number and returns the combined result using bitwise XOR
    """

    x = np.uint32(x)

    return np.uint32(ROTR(x, 6) ^ ROTR(x, 11) ^ ROTR(x, 25))

In [ ]:
def SHR(x, n):
    """
    Logical right shift on 32-bit value

    Shifts input right by n bits replaces vacated space with zeros
    """
    return np.uint32(x >> n)

In [ ]:
def sigma0(x):
    """
    sigma0 transformation to a 32-bit value

    Rotates input to the right by a fixed number, logical right shift and returns result using bitwise XOR
    """
    x = np.uint32(x)

    return np.uint32(ROTR(x, 7) ^ ROTR(x, 18) ^ SHR(x, 3))

In [ ]:
def sigma1(x):
    """
    sigma1 transformation to a 32-bit value

    Rotates input to the right by a fixed number, logical right shift and returns result using bitwise XOR
    """

    x = np.uint32(x)

    return np.uint32(ROTR(x, 17) ^ ROTR(x, 19) ^ SHR(x, 10))

Problem 2: Fractional Parts of Cube Roots

In [ ]:
def primes(n):
    """
    Generate the first of prime numbers

    Iteratively tests integers for whether its a prime number
    Returns a list containing the first n prime numbers
    """

    #Array to store prime numbers
    primeN = []
    #First number test if its a prime number
    num = 2
    #Until n number of prime numbers have been found
    while len(primeN) < n:
        #Assuming number is prime
        is_prime = True
        #Check if number can be divided by previous prime numbers
        for p in primeN:
            #If p * p is larger than num, stop checking
            if p * p > num:
                break
            #If number divided by p evenly it is not a prime number
            if num % p == 0:
                is_prime = False
                break
        #If number is still prime add to list
        if is_prime:
            primeN.append(num)
        #Move to next number
        num += 1
    #Return list of prime numbers
    return primeN

In [ ]:
def fractionalCubeRoots(primeNumbers):
    """
    Calculate 32-bit fractional parts of a cube roots of prime numbers

    extracts fractional part of each cube root, convert to 32bits and prints the value is hexadecimal format
    """
    #Store 32-bit values from fractional roots
    frac32 = []
    #Loop through each prime number
    for prime in primeNumbers:
        #Calculate cube root of prime number
        root = prime ** (1/3)
        #Store fractional part of cube root
        frac = root - int(root)
        #Convert the fractional result to a 32-bit integer
        bits = int(frac * (2**32))
        #Store the 32-bit value
        frac32.append(bits)

    #Print the values in a hexadecimal format
    for i, frac in enumerate(frac32):
        print(f"{frac:08x}", end=" ")
        if (i + 1) % 8 == 0:
            print()

In [3]:
primeNumbers = primes(64)
fractionalCubeRoots(primeNumbers)

428a2f98 71374491 b5c0fbcf e9b5dba5 3956c25b 59f111f1 923f82a4 ab1c5ed5 
d807aa98 12835b01 243185be 550c7dc3 72be5d74 80deb1fe 9bdc06a7 c19bf174 
e49b69c1 efbe4786 0fc19dc6 240ca1cc 2de92c6f 4a7484aa 5cb0a9dc 76f988da 
983e5152 a831c66d b00327c8 bf597fc7 c6e00bf3 d5a79147 06ca6351 14292967 
27b70a85 2e1b2138 4d2c6dfc 53380d13 650a7354 766a0abb 81c2c92e 92722c85 
a2bfe8a1 a81a664b c24b8b70 c76c51a3 d192e819 d6990624 f40e3585 106aa070 
19a4c116 1e376c08 2748774c 34b0bcb5 391c0cb3 4ed8aa4a 5b9cca4f 682e6ff3 
748f82ee 78a5636f 84c87814 8cc70208 90befffa a4506ceb bef9a3f7 c67178f2 


Problem 3: Padding

In [ ]:
def block_parse(msg: bytes):
    """
    Pad a message and split into a fixed size of blocks

    Apply padding to input and splits into 64-byte blocks
    """

    #Store message length in bits
    msgLenBits = len(msg) * 8
    #Copy message into mutable byte array
    paddedMsg = bytearray(msg)

    #Append a single "1" bit
    paddedMsg.append(0x80)

    #Add 0 bytes until the message length is 56 bytes mod 64
    while (len(paddedMsg) % 64) != (64-8):
            paddedMsg.append(0x00)

    #Append the original message length as a 64-bit big endian integer
    paddedMsg.extend(msgLenBits.to_bytes(8, "big"))

    #Split padded message into 64-byte blocks
    for blockStart in range(0, len(paddedMsg), 64):
        yield bytes(paddedMsg[blockStart:blockStart + 64])

In [6]:
msg = b'a'
for block in block_parse(msg):
    print(block.hex())

61800000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000008


Problem 4: Hashes

Problem 5: Passwords